In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import gensim.models
from gensim import models
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras import preprocessing
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding, LSTM
from keras.layers.recurrent import LSTM
from keras.models import Model, Sequential

# **IMPORT DATASET**

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/TA/dataset_new.csv')
dataset

,review_text,label
0,Hi my family and I are booked to travel to Bal...,1
1,"Hello Vishal, If you believe the latest statis...",1
2,"Hi Glenn, Thanks for your response. For me it ...",0
3,We are also travelling to Bali on the same day...,0
4,"On a positive note , I havent read of any Aust...",1
...,...,...
595,Are you in Australia experience with Dengeu Fe...,1
596,Singapore has zero deaths so far because of it...,1
597,While the rest of the 3rd world South East Asi...,1
598,"People want test in the first stage also, but ...",1


In [ ]:
dataset_no_prep = pd.read_csv('/content/drive/My Drive/TA/dataset_new.csv')

## pos neg

In [ ]:
pos = []
neg = []
for l in dataset.label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [ ]:
dataset['Pos']= pos
dataset['Neg']= neg

In [ ]:
dataset.tail()

,review_text,label,Pos,Neg
595,Are you in Australia experience with Dengeu Fe...,1,1,0
596,Singapore has zero deaths so far because of it...,1,1,0
597,While the rest of the 3rd world South East Asi...,1,1,0
598,"People want test in the first stage also, but ...",1,1,0
599,"Usually in the second stage from 8 persons, on...",1,1,0


# **PREPROCESSING**

## Data Cleansing (remove tanda baca)

Menghilangkan tanda baca.

In [ ]:
def cleansing(data):
  removed_dot = []
  for review in data:
    removed_dot.append(review.replace('.', ' '))
  data = removed_dot

  removed_koma = []
  for review in data:
    removed_koma.append(review.replace(',', ' '))
  data = removed_koma

  removed_slash = []
  for review in data:
    removed_slash.append(review.replace('/', ' '))
  data = removed_slash
  
  removed_dan = []
  for review in data:
    removed_dan.append(review.replace('&', ' '))
  data = removed_dan

  removed_brack = []
  for review in data:
    removed_brack.append(review.replace('(', ' '))
  data = removed_brack

  removed_brack1 = []
  for review in data:
    removed_brack1.append(review.replace(')', ' '))
  data = removed_brack1

  removed_strip = []
  for review in data:
    removed_strip.append(review.replace('-', ' '))
  data = removed_strip
  
  removed_q = []
  for review in data:
    removed_q.append(review.replace('?', ' '))
  data = removed_q
  
  removed_qt = []
  for review in data:
    removed_qt.append(review.replace('"', ' '))
  data = removed_qt
  
  removed_col = []
  for review in data:
    removed_col.append(review.replace(':', ' '))
  data = removed_col

  removed_per = []
  for review in data:
    removed_per.append(review.replace('%', ' '))
  data = removed_per

  removed_plus = []
  for review in data:
    removed_plus.append(review.replace('+', ' '))
  data = removed_plus

  removed_dol = []
  for review in data:
    removed_dol.append(review.replace('$', ' '))
  data = removed_dol

  removed_exc = []
  for review in data:
    removed_exc.append(review.replace('!', ' '))
  data = removed_exc

  removed_tag = []
  for review in data:
    removed_tag.append(review.replace('#', ' '))
  data = removed_tag

  removed_eq = []
  for review in data:
    removed_eq.append(review.replace('=', ' '))
  data = removed_eq

  return data

Ubah kata dengan ' sebagai apostof menjadi kata yang sebenarnya.

In [ ]:
def apostrof(data):
  removed_1 = []
  for review in data:
    removed_1.append(review.replace("I’m", "I am"))
  data= removed_1

  removed_2 = []
  for review in data:
    removed_2.append(review.replace("it’s", "it is"))
  data = removed_2

  removed_3 = []
  for review in data:
    removed_3.append(review.replace("doesn’t", "does not"))
  data = removed_3

  removed_4 = []
  for review in data:
    removed_4.append(review.replace("I’ve", "I have"))
  data = removed_4

  removed_5 = []
  for review in data:
    removed_5.append(review.replace("don’t", "do not"))
  data = removed_5

  removed_6 = []
  for review in data:
    removed_6.append(review.replace("there’s", "there is"))
  data = removed_6

  removed_7 = []
  for review in data:
    removed_7.append(review.replace("that’s", "that is"))
  data = removed_7

  removed_8 = []
  for review in data:
    removed_8.append(review.replace("didn’t", "did not"))
  data = removed_8

  removed_9 = []
  for review in data:
    removed_9.append(review.replace("wasn’t", "was not"))
  data = removed_9

  removed_10 = []
  for review in data:
    removed_10.append(review.replace("I’d", "I would"))
  data = removed_10

  removed_11 = []
  for review in data:
    removed_11.append(review.replace("’s", " "))
  data = removed_11

  removed_12 = []
  for review in data:
    removed_12.append(review.replace("I’ll", "I will"))
  data = removed_12

  removed_13 = []
  for review in data:
    removed_13.append(review.replace("wouldn’t", "would not"))
  data = removed_13

  removed_14 = []
  for review in data:
    removed_14.append(review.replace("aren’t", "are not"))
  data = removed_14

  removed_15 = []
  for review in data:
    removed_15.append(review.replace("shouldn’t", "should not"))
  data = removed_15

  removed_16 = []
  for review in data:
    removed_16.append(review.replace("you’re", "you are"))
  data = removed_16

  removed_17 = []
  for review in data:
    removed_17.append(review.replace("we’re", "we are"))
  data = removed_17

  removed_18 = []
  for review in data:
    removed_18.append(review.replace("can’t", "can not"))
  data = removed_18

  removed_19 = []
  for review in data:
    removed_19.append(review.replace("isn’t", "is not"))
  data = removed_19

  removed_20 = []
  for review in data:
    removed_20.append(review.replace("haven’t", "have not"))
  data = removed_20

  removed_21 = []
  for review in data:
    removed_21.append(review.replace("couldn’t", "could not"))
  data = removed_21

  removed_22 = []
  for review in data:
    removed_22.append(review.replace("what’s", "what is"))
  data = removed_22

  removed_23 = []
  for review in data:
    removed_23.append(review.replace("he’s", "he is"))
  data = removed_23

  removed_24 = []
  for review in data:
    removed_24.append(review.replace("weren’t", "were not"))
  data = removed_24

  removed_25 = []
  for review in data:
    removed_25.append(review.replace("it’ll", "it will"))
  data = removed_25

  removed_26 = []
  for review in data:
    removed_26.append(review.replace("we’ve", "we have"))
  data = removed_26

  removed_27 = []
  for review in data:
    removed_27.append(review.replace("won’t", "will not"))
  data = removed_27

  removed_28 = []
  for review in data:
    removed_28.append(review.replace("how’s", "how is"))
  data = removed_28

  removed_29 = []
  for review in data:
    removed_29.append(review.replace("havent", "have not"))
  data = removed_29

  removed_petik = []
  for review in data:
    removed_petik.append(review.replace("’", " "))
  data = removed_petik

  removed_0 = []
  for review in data:
    removed_0.append(review.replace("0", ""))
  data = removed_0

  removed_1 = []
  for review in data:
    removed_1.append(review.replace("1", ""))
  data = removed_1

  removed_2 = []
  for review in data:
    removed_2.append(review.replace("2", ""))
  data = removed_2

  removed_3 = []
  for review in data:
    removed_3.append(review.replace("3", ""))
  data = removed_3

  removed_4 = []
  for review in data:
    removed_4.append(review.replace("4", ""))
  data = removed_4

  removed_5 = []
  for review in data:
    removed_5.append(review.replace("5", ""))
  data = removed_5

  removed_6 = []
  for review in data:
    removed_6.append(review.replace("6", ""))
  data = removed_6

  removed_7 = []
  for review in data:
    removed_7.append(review.replace("7", ""))
  data = removed_7

  removed_8 = []
  for review in data:
    removed_8.append(review.replace("8", ""))
  data = removed_8

  removed_9 = []
  for review in data:
    removed_9.append(review.replace("9", ""))
  data = removed_9

  return data

In [ ]:
dat = apostrof(cleansing(dataset['review_text']))
print(dat)

['Hi my family and I are booked to travel to Bali via Thailand on the th of March  I will be travelling with my parents and my young children        I was wondering if it was safe to go to Bali at this time and whether there being zero reported Corona Virus cases was true for Bali  The company that I have booked my villa through have been following up with me for my balance payment and assure me that Bali is safe but it would be great to hear from other travellers that are planning to visit or already there ', 'Hello Vishal  If you believe the latest statistics being  cases in India   cases in Thailand and still none in Bali  You are safer in Bali than anywhere else  I am in Bali atm and life is as normal here ', 'Hi Glenn  Thanks for your response  For me it was a bigger concern when I read that Bali Indonesia has zero cases as the virus has reached parts of Europe as well which is much further away   is it really true  Some say that it is because they do not have test kits over there

## Stopword Removal

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def stop(dat):
  stop_words = set(stopwords.words('english')) 

  for i,v in enumerate(dat):
    words = word_tokenize(v)
    
    text=''
    for w in words: 
        if w not in stop_words:
          text=text+' '+w
          dat[i]=text
    del words
  return dat

dat = stop(dat)

## Lower Casing

In [ ]:
data = []
for d in dat:
  data.append(d.lower())

## Tokenization

In [ ]:
tokenization = []
for review in data:
  tokenization.append(word_tokenize(review))

# print(tokenization)

[['hi', 'family', 'i', 'booked', 'travel', 'bali', 'via', 'thailand', 'th', 'march', 'i', 'travelling', 'parents', 'young', 'children', 'i', 'wondering', 'safe', 'go', 'bali', 'time', 'whether', 'zero', 'reported', 'corona', 'virus', 'cases', 'true', 'bali', 'the', 'company', 'i', 'booked', 'villa', 'following', 'balance', 'payment', 'assure', 'bali', 'safe', 'would', 'great', 'hear', 'travellers', 'planning', 'visit', 'already'], ['hello', 'vishal', 'if', 'believe', 'latest', 'statistics', 'cases', 'india', 'cases', 'thailand', 'still', 'none', 'bali', 'you', 'safer', 'bali', 'anywhere', 'else', 'i', 'bali', 'atm', 'life', 'normal'], ['hi', 'glenn', 'thanks', 'response', 'for', 'bigger', 'concern', 'i', 'read', 'bali', 'indonesia', 'zero', 'cases', 'virus', 'reached', 'parts', 'europe', 'well', 'much', 'away', 'really', 'true', 'some', 'say', 'test', 'kits', 'others', 'say', 'hidden', 'hit', 'tourism', 'i', 'hope', 'safe', 'trip', 'bali', 'share', 'updates', 'hear', 'anything', 'thank

## Stemming

In [ ]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

In [ ]:
def stemming(data):
  stem_data = []
  for review in data:
    stemm = []
    for word in review:
      stemm.append(porter.stem(word))
      # stem_data.append(" ")
    stem_data.append(stemm)
  return stem_data
  

stemming_tokens = stemming(tokenization)
stemming_text = stemming(data)
# print(stemming_tokens)

[['hi', 'famili', 'i', 'book', 'travel', 'bali', 'via', 'thailand', 'th', 'march', 'i', 'travel', 'parent', 'young', 'children', 'i', 'wonder', 'safe', 'go', 'bali', 'time', 'whether', 'zero', 'report', 'corona', 'viru', 'case', 'true', 'bali', 'the', 'compani', 'i', 'book', 'villa', 'follow', 'balanc', 'payment', 'assur', 'bali', 'safe', 'would', 'great', 'hear', 'travel', 'plan', 'visit', 'alreadi'], ['hello', 'vishal', 'if', 'believ', 'latest', 'statist', 'case', 'india', 'case', 'thailand', 'still', 'none', 'bali', 'you', 'safer', 'bali', 'anywher', 'els', 'i', 'bali', 'atm', 'life', 'normal'], ['hi', 'glenn', 'thank', 'respons', 'for', 'bigger', 'concern', 'i', 'read', 'bali', 'indonesia', 'zero', 'case', 'viru', 'reach', 'part', 'europ', 'well', 'much', 'away', 'realli', 'true', 'some', 'say', 'test', 'kit', 'other', 'say', 'hidden', 'hit', 'tourism', 'i', 'hope', 'safe', 'trip', 'bali', 'share', 'updat', 'hear', 'anyth', 'thank'], ['we', 'also', 'travel', 'bali', 'day', 'via', '

In [ ]:
result = [' '.join(sen) for sen in stemming_tokens]

In [ ]:
dataset['tokens'] = stemming_tokens
dataset['text_final'] = result

# **SPLIT DATASET INTO TRAIN AND TEST**

In [ ]:
dataset = dataset[['text_final', 'tokens', 'Pos', 'Neg', 'label']]
dataset[:4]

,text_final,tokens,Pos,Neg,label
0,hi famili i book travel bali via thailand th m...,"[hi, famili, i, book, travel, bali, via, thail...",1,0,1
1,hello vishal if believ latest statist case ind...,"[hello, vishal, if, believ, latest, statist, c...",1,0,1
2,hi glenn thank respons for bigger concern i re...,"[hi, glenn, thank, respons, for, bigger, conce...",0,1,0
3,we also travel bali day via bangkok realli wor...,"[we, also, travel, bali, day, via, bangkok, re...",0,1,0


In [ ]:
dataset.shape

(600, 5)

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(dataset, test_size=0.2, shuffle=False)

In [ ]:
from sklearn.model_selection import train_test_split
train_no_prep, test_no_prep = train_test_split(dataset_no_prep, test_size=0.2, shuffle=False)

# **FEATURE EXTRACTION / WORD EMBEDDING (Word2Vec)**

Load Word2Vec Model
300 Dimensi
Skipgram
Negative sampling

In [ ]:
word2vec_300 = KeyedVectors.load_word2vec_format("drive/My Drive/TA/ws_model300.txt")

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    # else:
    #     vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

In [ ]:
def get_word2vec_embeddings(vectors, dataset, generate_missing=False):
    embeddings = dataset.apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
    return list(embeddings)

Get Embeddings

In [ ]:
training_embeddings = get_word2vec_embeddings(word2vec_300, data_train['tokens'], generate_missing=True)
testing_embeddings = get_word2vec_embeddings(word2vec_300, data_test['tokens'], generate_missing=True)

In [ ]:
training_no_prep = get_word2vec_embeddings(word2vec_300, train_no_prep['review_text'], generate_missing=True)
testing_no_prep = get_word2vec_embeddings(word2vec_300, test_no_prep['review_text'], generate_missing=True)

In [ ]:
MAX_SEQUENCE_LENGTH =  50

EMBEDDING_DIM = 300


## Pad Sequence

 X_TRAIN

In [ ]:
tokenizer = Tokenizer(num_words=len(training_embeddings))
tokenizer.fit_on_texts(data_train["tokens"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["tokens"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
print(len(training_sequences))

x_train = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 2816 unique tokens.
480


In [ ]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec_300[word] if word in word2vec_300 else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(2817, 300)


Y_TRAIN

In [ ]:
label_names = ['Pos', 'Neg']
y_train = data_train[label_names].values

X_TRAIN NO PREP

In [ ]:
tokenizer = Tokenizer(num_words=len(training_no_prep))
tokenizer.fit_on_texts(train_no_prep['review_text'].tolist())
training_sequences = tokenizer.texts_to_sequences(train_no_prep['review_text'].tolist())

train_np_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_np_word_index))
print(len(training_sequences))

train_no_prep = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 4023 unique tokens.
480


In [ ]:
train_np_embedding_weights = np.zeros((len(train_np_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_np_embedding_weights[index,:] = word2vec_300[word] if word in word2vec_300 else np.random.rand(EMBEDDING_DIM)
print(train_np_embedding_weights.shape)

(4024, 300)


X_TEST

In [ ]:
tokenizer = Tokenizer(num_words=len(testing_embeddings))
tokenizer.fit_on_texts(data_test["tokens"].tolist())
testing_sequences = tokenizer.texts_to_sequences(data_test["tokens"].tolist())

test_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(test_word_index))
print(len(testing_sequences))

x_test = pad_sequences(testing_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 1147 unique tokens.
120


In [ ]:
y_test = data_test['label'].values

X_TEST NO PREP

In [ ]:
tokenizer = Tokenizer(num_words=len(testing_no_prep))
tokenizer.fit_on_texts(test_no_prep['review_text'].tolist())
testing_sequences = tokenizer.texts_to_sequences(test_no_prep['review_text'].tolist())

test_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(test_word_index))
print(len(testing_sequences))

test_no_prep = pad_sequences(testing_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 1516 unique tokens.
120


SAVE DATA

In [ ]:
from numpy import savetxt

savetxt("drive/My Drive/TA/data_train_no_prep.csv", train_no_prep, delimiter=",")
savetxt("drive/My Drive/TA/data_test_no_prep.csv", test_no_prep, delimiter=",")

In [ ]:
from numpy import savetxt

savetxt("drive/My Drive/TA/data_train.csv", x_train, delimiter=",")
savetxt("drive/My Drive/TA/y_train.csv", y_train, delimiter=",")
savetxt("drive/My Drive/TA/data_test.csv", x_test, delimiter=",")
savetxt("drive/My Drive/TA/y_test.csv", y_test, delimiter=",")

# **LSTM**

## TRAIN LSTM

In [ ]:
  def lstm(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words, embedding_dim, weights=[embeddings], input_length=max_sequence_length, trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

#     lstm = LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()
    return model

In [ ]:
label_names = ['Pos', 'Neg']

In [ ]:
model = lstm(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

SAVE MODEL

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


## TEST LSTM

LOAD MODEL

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('/content/drive/My Drive/TA/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/TA/model.h5")
print("Loaded model from disk")

Loaded model from disk


LOAD DATA

In [ ]:
from numpy import loadtxt

x_train = loadtxt("drive/My Drive/TA/data_train.csv", delimiter=',')
y_train = loadtxt("drive/My Drive/TA/y_train.csv", delimiter=',')
x_test = loadtxt("drive/My Drive/TA/data_test.csv", delimiter=',')
y_test = loadtxt("drive/My Drive/TA/y_test.csv", delimiter=',')
# x_train = loadtxt("drive/My Drive/TA/data_train_no_prep.csv", delimiter=',')
# x_test = loadtxt("drive/My Drive/TA/data_test_no_prep.csv", delimiter=',')

In [ ]:
import keras

class_weight = {1: 0.50,
                0: 0.95}

num_epochs = 5
batch_size = 34

In [ ]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
loaded_model.fit(x_train, y_train, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size, class_weight=class_weight)

LOAD PREDICTIONS

In [ ]:
predictions = loaded_model.predict(x_test, batch_size=batch_size, verbose=10)

In [ ]:
labels = [1, 0]

prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

pred = prediction_labels

In [ ]:
from numpy import savetxt
#SAVE PREDICTIONS

#savetxt("drive/My Drive/TA/predictions1.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions2.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions3.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions4.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions5.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions6.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions7.csv", prediction_labels, delimiter=",")
# savetxt("drive/My Drive/TA/predictions8.csv", prediction_labels, delimiter=",")

LOAD DATA PREDICTIONS SELURUH SKENARIO

In [ ]:
from numpy import loadtxt
#LOAD PREDICTIONS
pred = loadtxt("drive/My Drive/TA/predictions1.csv", delimiter=',')
pred2 = loadtxt("drive/My Drive/TA/predictions2.csv", delimiter=',')
pred3 = loadtxt("drive/My Drive/TA/predictions3.csv", delimiter=',')
pred4 = loadtxt("drive/My Drive/TA/predictions4.csv", delimiter=',')
pred5 = loadtxt("drive/My Drive/TA/predictions5.csv", delimiter=',')
pred6 = loadtxt("drive/My Drive/TA/predictions6.csv", delimiter=',')
pred7 = loadtxt("drive/My Drive/TA/predictions7.csv", delimiter=',')
pred8 = loadtxt("drive/My Drive/TA/predictions8.csv", delimiter=',')

# HASIL EVALUASI

HASIL AKURASI SELURUH SKENARIO


In [ ]:
def accuracy (y_test, pred):
    miss_pred = np.sum(np.not_equal(np.array(y_test), np.array(pred)))/float(y_test.size)
    accuracy = 1-miss_pred
    return round((accuracy * 100),2)

print(accuracy(y_test,pred))

60.83


In [ ]:
print('Accuracy skenario I : ',accuracy(y_test, pred),'%')
print('Accuracy skenario II : ',accuracy(y_test, pred2),'%')
print('Accuracy skenario III : ',accuracy(y_test, pred3),'%')
print('Accuracy skenario IV : ',accuracy(y_test, pred4),'%')
print('Accuracy skenario V : ',accuracy(y_test, pred5),'%')
print('Accuracy skenario VI : ',accuracy(y_test, pred6),'%')
print('Accuracy skenario VII : ',accuracy(y_test, pred7),'%')
print('Accuracy skenario VIII : ',accuracy(y_test, pred8),'%')

Accuracy skenario I :  55.83 %
Accuracy skenario II :  60.83 %
Accuracy skenario III :  60.0 %
Accuracy skenario IV :  61.67 %
Accuracy skenario V :  65.0 %
Accuracy skenario VI :  64.17 %
Accuracy skenario VII :  71.67 %
Accuracy skenario VIII :  65.83 %


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

HASIL PRECISION SELURUH SKENARIO

In [ ]:
def precision(y_test, pred):
  prec = precision_score(y_test, pred)
  return round((prec * 100), 2)

In [ ]:
print('Precision skenario I : ',precision(y_test, pred),'%')
print('Precision skenario II : ',precision(y_test, pred2),'%')
print('Precision skenario III : ',precision(y_test, pred3),'%')
print('Precision skenario IV : ',precision(y_test, pred4),'%')
print('Precision skenario V : ',precision(y_test, pred5),'%')
print('Precision skenario VI : ',precision(y_test, pred6),'%')  
print('Precision skenario VII : ',precision(y_test, pred7),'%')
print('Precision skenario VIII : ',precision(y_test, pred8),'%')

Precision skenario I :  70.51 %
Precision skenario II :  73.17 %
Precision skenario III :  72.29 %
Precision skenario IV :  73.49 %
Precision skenario V :  75.29 %
Precision skenario VI :  75.0 %
Precision skenario VII :  72.97 %
Precision skenario VIII :  75.58 %


HASIL RECALL SELURUH SKENARIO

In [ ]:
def recall(y_test, pred):
  rec = recall_score(y_test, pred)
  return round((rec * 100), 1)

In [ ]:
print('Recall skenario I : ', recall(y_test, pred),'%')
print('Recall skenario II : ', recall(y_test, pred2),'%')
print('Recall skenario III : ', recall(y_test, pred3),'%')
print('Recall skenario IV : ', recall(y_test, pred4),'%')
print('Recall skenario V : ', recall(y_test, pred5),'%')
print('Recall skenario VI : ', recall(y_test, pred6),'%')
print('Recall skenario VII : ', recall(y_test, pred7),'%')
print('Recall skenario VIII : ', recall(y_test, pred8),'%')

Recall skenario I :  64.7 %
Recall skenario II :  70.6 %
Recall skenario III :  70.6 %
Recall skenario IV :  71.8 %
Recall skenario V :  75.3 %
Recall skenario VI :  74.1 %
Recall skenario VII :  95.3 %
Recall skenario VIII :  76.5 %


HASIL F1-SCORE SELURUH SKENARIO

In [ ]:
def f1score(y_test, pred):
  f1 = f1_score(y_test, pred)
  return round((f1 * 100), 1)

In [ ]:
print('F1-Score Skenario I : ',f1score(y_test,pred),'%')
print('F1-Score Skenario II : ',f1score(y_test,pred2),'%')
print('F1-Score Skenario III : ',f1score(y_test,pred3),'%')
print('F1-Score Skenario IV : ',f1score(y_test,pred4),'%')
print('F1-Score Skenario V : ',f1score(y_test,pred5),'%')
print('F1-Score Skenario VI : ',f1score(y_test,pred6),'%')
print('F1-Score Skenario VII : ',f1score(y_test,pred7),'%')
print('F1-Score Skenario VIII : ',f1score(y_test,pred8),'%')

F1-Score Skenario I :  67.5 %
F1-Score Skenario II :  71.9 %
F1-Score Skenario III :  71.4 %
F1-Score Skenario IV :  72.6 %
F1-Score Skenario V :  75.3 %
F1-Score Skenario VI :  74.6 %
F1-Score Skenario VII :  82.7 %
F1-Score Skenario VIII :  76.0 %


CONFUSION MATRIX SKENARIO 7 (hasil akurasi tertinggi)

In [ ]:
print('Matrix : \n', confusion_matrix(y_test, pred7))

Matrix : 
 [[ 5 30]
 [ 4 81]]
